
---

# 📌 DataLoader 정리

## 1. DataLoader란?

* `torch.utils.data.DataLoader`는 **Dataset + Sampler**를 결합해서
  → **반복 가능한 객체(iterable)**로 만들어주는 클래스.
* 역할:

  * 데이터를 **batch 단위**로 묶어줌
  * epoch마다 **데이터 순서를 섞어줌(shuffle)**
  * **병렬 프로세싱(num_workers)**으로 빠르게 데이터 로딩
  * GPU 학습 시 **pin_memory**로 전송 최적화

---

## 2. 주요 인자

### 필수

* **`dataset`**
  어떤 데이터를 읽을지 (`TensorDataset`, `ImageFolder`, `MNIST`, 커스텀 Dataset 등)

### 자주 쓰는 핵심 인자

* **`batch_size`** (기본 1) → 한 번에 몇 개의 샘플을 반환할지
* **`shuffle`** (기본 False)

  * `True`: epoch마다 섞음 → **train** 데이터셋에 주로 사용
  * `False`: 섞지 않음 → **validation/test** 데이터셋에 사용
* **`num_workers`** (기본 0) → 데이터 로딩에 사용할 CPU 프로세스 수


```python
# cpu 코어 확인
import os
  os.cpu_count()

```

  * 0: 메인 프로세스 (느림, 디버깅용)
  * > 0: 병렬 처리 (빠름, 실전용)
* **`drop_last`** (기본 False)

  * 데이터 개수가 `batch_size`로 나누어떨어지지 않을 때, 마지막 배치 버릴지 여부

### 상황에 따라 중요한 인자

* **`collate_fn`** → batch로 묶는 방식을 정의 (특히 NLP, variable-length 데이터 처리에 중요)
* **`pin_memory`** → GPU 학습 시 데이터 전송 최적화 (`True` 권장)
* **`generator`** → 셔플 시 시드 고정 (재현성 확보)
* **`persistent_workers`** → epoch 끝나도 worker를 종료하지 않고 유지 (속도 최적화)

---

## 3. 동작 과정

1. DataLoader는 dataset에서 **샘플 인덱스**를 가져옴 (sampler)
2. batch_size만큼 모아서 배치(batch)를 만듦
3. collate_fn으로 텐서 묶음으로 변환
4. GPU 학습이면 pin_memory로 메모리 고정 후 반환
5. epoch이 끝나면 다시 shuffle 가능

---

## 4. 사용 예시

```python
from torch.utils.data import DataLoader, TensorDataset
import torch

# 데이터셋 만들기
data = torch.arange(20).reshape(10, 2).float()
labels = torch.arange(10)
dataset = TensorDataset(data, labels)

# DataLoader 생성
loader = DataLoader(dataset,
                    batch_size=4,
                    shuffle=True,
                    num_workers=2,
                    drop_last=False,
                    pin_memory=True)

# 반복 사용
for epoch in range(2):
    print(f"Epoch {epoch+1}")
    for batch in loader:
        x, y = batch
        print(x, y)
```

---

## 5. 보통 설정하는 방식

* **train loader**

  ```python
  DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, drop_last=True, pin_memory=True)
  ```
* **validation/test loader**

  ```python
  DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=4, drop_last=False, pin_memory=True)
  ```

---

## ✅ 핵심 요약

* **Dataset → DataLoader**로 감싸야 모델 학습 루프에서 쉽게 사용 가능
* 꼭 알아야 할 인자: `dataset`, `batch_size`, `shuffle`, `num_workers`, `drop_last`
* 성능 최적화: `pin_memory=True`, `num_workers`를 CPU 코어 개수에 맞춰 튜닝

---

현태, 내가 DataLoader를 **train/valid/test로 나누어서 실제 학습 루프(`for epoch in range...`)**에 어떻게 들어가는지 예제 코드도 정리해줄까?



---

## 📌 1. `collate_fn`

* 역할: **여러 샘플을 하나의 배치(batch)로 묶는 방식**을 정의.
* 기본 동작: 리스트로 가져온 샘플들을 `torch.stack()`해서 텐서로 합쳐줌.
* 필요할 때:

  * 데이터 길이가 제각각일 때 (예: NLP 문장 길이 다름 → 패딩 필요)
  * 배치 단위 전처리를 하고 싶을 때
  * 커스텀 Dataset을 쓸 때 데이터 형태가 텐서가 아닐 경우

👉 예시 (문장의 길이가 다를 때 패딩):



In [8]:
from torch.utils.data import DataLoader, TensorDataset
import torch

# 데이터셋 만들기
data = torch.arange(20).reshape(10, 2).float()
labels = torch.arange(10)
dataset = TensorDataset(data, labels)

# DataLoader 생성
def collate_batch(batch):
    # batch = [샘플1, 샘플2, ...] (각 샘플은 텍스트 길이 다름)
    texts, labels = zip(*batch)
    # 가장 긴 문장 길이에 맞춰 패딩
    max_len = max(len(t) for t in texts)
    padded = [t + [0]*(max_len-len(t)) for t in texts]
    return torch.tensor(padded), torch.tensor(labels)

loader = DataLoader(dataset, batch_size=4, collate_fn=collate_batch)



---

## 📌 2. `pin_memory`

* 역할: **데이터를 CUDA pinned memory(고정 메모리)**에 올려둔 뒤 GPU로 바로 전송.
* 왜 필요해?

  * 일반적으로 CPU→GPU 전송은 비동기 처리가 안 되는데,
  * pinned memory를 쓰면 **비동기 전송**이 가능해서 속도가 빨라짐.
* GPU 학습 시 `pin_memory=True` 권장 (특히 이미지/대규모 텐서).

---

---

## 📌 동기(Synchronous)

* CPU가 GPU로 데이터를 보낼 때,
  **GPU가 다 받았는지 확인할 때까지 CPU가 기다림**
* 즉, CPU와 GPU가 한 번에 하나의 일만 순서대로 처리 → 병목 가능

---

## 📌 비동기(Asynchronous)

* CPU가 GPU로 데이터를 보내는 “명령”만 내리고,
  **GPU가 받는 동안 CPU는 다른 작업을 계속함**
* CPU와 GPU가 동시에 일을 처리할 수 있음 → 속도 최적화

---

## 📌 PyTorch에서

* 그냥 `.to("cuda")` → 동기 전송
* `pin_memory=True` + `.to("cuda", non_blocking=True)` → 비동기 전송 가능

---

✅ 한 줄 요약:
동기 = CPU가 GPU 기다림
비동기 = CPU와 GPU가 동시에 일함 (pin_memory + non_blocking 필요)

---



In [10]:
# pin_memory= True
loader = DataLoader(dataset, batch_size=64, shuffle=True, pin_memory=True)
for data, target in loader:
    data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)



---

## 📌 3. `generator`

* 역할: 난수 생성기 지정. (`torch.Generator`)
* 언제 쓰냐?

  * `shuffle=True`일 때 데이터 순서를 섞는 난수를 제어
  * `random_split` 등과 동일하게, **seed 고정**해서 **재현성 확보**할 때 필요
* 예시:

```python
g = torch.Generator().manual_seed(42)
loader = DataLoader(dataset, batch_size=32, shuffle=True, generator=g)
```

→ 실행할 때마다 항상 같은 순서로 섞임.

---



In [9]:
# Generator().manual_seed(42)
g = torch.Generator().manual_seed(42)
loader = DataLoader(dataset, batch_size=32, shuffle=True, generator=g)



---

## 📌 4. `persistent_workers`

* 역할: DataLoader에서 worker 프로세스를 **epoch이 끝난 뒤에도 종료하지 않고 유지**.
* 왜 필요해?

  * 기본값(`False`) → epoch이 끝날 때마다 worker가 종료됐다가 다시 만들어짐 → overhead 발생
  * `True` → worker 유지 → 다음 epoch에서 바로 사용 → 속도 향상
* 조건: `num_workers > 0`일 때만 의미 있음.
* 긴 학습에서 **성능 최적화**용으로 쓰임.

---

In [ ]:
loader = DataLoader(dataset, batch_size=64, num_workers=4, persistent_workers=True)